In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"  
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from keras.callbacks import TensorBoard
from TrainValTensorBoard import TrainValTensorBoard
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())
print(tf.test.is_built_with_cuda())

False


In [5]:
df_train = pd.read_csv('../train.csv')
df_test = pd.read_csv('../test.csv')

In [6]:
df_test

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
def preprocess_data(df_train):
    df_train_rand = df_train.reindex(np.random.permutation(df_train.index))
    return df_train_rand

In [8]:
def preprocess_features(x_train):
    #Normalization to greyscale, reduces the effect of illumination's differences
    x_train_norm = x_train / 255.0
    #Reshaping for keras 28x28x1 3D matrices?
    x_train_norm = x_train_norm.values.reshape(-1, 28, 28, 1)
    return x_train_norm

In [9]:
def preprocess_targets(y_train):
    #Encode labels to one hot vecs
    y_train_hot = tf.keras.utils.to_categorical(y_train, num_classes=10)
    return y_train_hot

In [10]:
df_train_rand = preprocess_data(df_train)
X_train = preprocess_features(df_train_rand.drop(labels = ['label'], axis = 1).head(38000).copy())
X_val = preprocess_features(df_train_rand.drop(labels = ['label'], axis = 1).tail(6000).copy())
Y_train = preprocess_targets(df_train_rand['label'].head(38000).copy())
Y_val = preprocess_targets(df_train_rand['label'].tail(6000).copy())

In [11]:
visible = tf.keras.layers.Input(shape=(28,28,1))

In [12]:
conv1 = tf.keras.layers.Conv2D(32, kernel_size=(5,5), padding='Same', activation='relu')(visible)
conv2 = tf.keras.layers.Conv2D(32, kernel_size=(5,5), padding='Same', activation='relu')(conv1)
pool1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2))(conv2)
dropout1 = tf.keras.layers.Dropout(0.25)(pool1)
conv3 = tf.keras.layers.Conv2D(64, kernel_size=(3,3), padding='Same', activation='relu')(dropout1)
conv4 = tf.keras.layers.Conv2D(64, kernel_size=(3,3), padding='Same', activation='relu')(conv3)
pool2 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2))(conv4)
dropout2 = tf.keras.layers.Dropout(0.25)(pool2)
flatten = tf.keras.layers.Flatten()(dropout2)
dense1 = tf.keras.layers.Dense(245, activation='relu')(flatten)
dropout3 = tf.keras.layers.Dropout(0.25)(dense1)
output_dense = tf.keras.layers.Dense(10, activation='softmax')(dropout3)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
model = tf.keras.Model(inputs=visible, outputs=output_dense)

In [14]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001,
                                   beta1 = 0.9,
                                   beta2 = 0.999,
                                   epsilon = 1e-08,
                                   use_locking = False,
                                  name = 'Adam')

In [15]:
# Compiling
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [16]:
tensorboardg = TrainValTensorBoard()

In [17]:
#tensorboard_j = tf.keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

In [18]:
tf.test.is_gpu_available()

False

In [19]:
history = model.fit(X_train,
                   Y_train,
                   batch_size=80,
                   epochs=10,
                   validation_data=(X_val, Y_val),
                   verbose=2,
                   callbacks=[TrainValTensorBoard(write_graph=False)])

Train on 38000 samples, validate on 6000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
 - 107s - loss: 0.2037 - acc: 0.9354 - val_loss: 0.0781 - val_acc: 0.9753
Epoch 2/10
 - 104s - loss: 0.0649 - acc: 0.9800 - val_loss: 0.0412 - val_acc: 0.9868
Epoch 3/10
 - 103s - loss: 0.0484 - acc: 0.9849 - val_loss: 0.0427 - val_acc: 0.9865
Epoch 4/10
 - 101s - loss: 0.0382 - acc: 0.9880 - val_loss: 0.0301 - val_acc: 0.9907
Epoch 5/10
 - 100s - loss: 0.0326 - acc: 0.9895 - val_loss: 0.0270 - val_acc: 0.9913
Epoch 6/10
 - 100s - loss: 0.0304 - acc: 0.9904 - val_loss: 0.0217 - val_acc: 0.9928
Epoch 7/10
 - 158s - loss: 0.0240 - acc: 0.9924 - val_loss: 0.0229 - val_acc: 0.9933
Epoch 8/10
 - 126s - loss: 0.0229 - acc: 0.9927 - val_loss: 0.0223 - val_acc: 0.9928
Epoch 9/10
 - 102s - loss: 0.0208 - acc: 0.9938 - val_loss: 0.0274 - val_acc: 0.9928
Epoch 10/10
 - 100s - loss: 0.0180 - acc: 0.9944 - val_loss: 0.0178 - val_acc: 0.9945


In [16]:
x_test = preprocess_features(df_test)
x_test.shape

(28000, 28, 28, 1)

In [17]:
predictions = model.predict(x_test)
predictions.shape

(28000, 10)

In [18]:
results = np.argmax(predictions, axis=1)
results_S = pd.Series(results, name='Label')
results_S

0        2
1        0
2        9
3        9
4        3
5        7
6        0
7        3
8        0
9        3
10       5
11       7
12       4
13       0
14       4
15       3
16       3
17       1
18       9
19       0
20       9
21       1
22       1
23       5
24       7
25       4
26       2
27       7
28       4
29       7
        ..
27970    5
27971    0
27972    4
27973    8
27974    0
27975    3
27976    6
27977    0
27978    1
27979    9
27980    3
27981    1
27982    1
27983    0
27984    4
27985    5
27986    2
27987    2
27988    9
27989    6
27990    7
27991    6
27992    1
27993    9
27994    7
27995    9
27996    7
27997    3
27998    9
27999    2
Name: Label, Length: 28000, dtype: int64

In [19]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results_S],axis = 1)
submission

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
5,6,7
6,7,0
7,8,3
8,9,0
9,10,3


In [20]:
submission.to_csv("predictions_cnn_keras_func_mnist.csv",index=False)